<b>Q1. Running Elastic</b>

In [1]:
import requests
import minsearch

In [ ]:
# !pip install scikit-learn

In [36]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [ ]:
# documents

In [3]:
index = minsearch.Index(
    text_fields = ["question","text","section"],
    keyword_fields = ["course"]
)

In [4]:
index

In [5]:
index.fit(documents)

In [6]:
from openai import OpenAI

In [7]:
client = OpenAI()

In [8]:
def search(query):
    boost = {'question': 3.0, 'section':0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=10
    )
    return results

In [9]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION: {question}
    
    CONTEXT:
    {context}
    """.strip()
    
    context = ""

    for doc in search_results:
        context = context + f"section:  {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n" 

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [10]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user","content":prompt}]
    )
    return response.choices[0].message.content

In [11]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [12]:
from elasticsearch import Elasticsearch

In [13]:
es_client = Elasticsearch('http://localhost:9200')

In [19]:
# es_client.info()

ObjectApiResponse({'name': '29ff33a96aeb', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'TeQnW9CWR6ePM29DnKWtlQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [40]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}
index_name = "course-questions"
# es_client.indices.create(index=index_name, body=index_settings)

In [16]:
from tqdm.auto import tqdm

/home/codespace/.local/share/virtualenvs/llm-zoomcamp-workshop-H1KjvqNQ/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:24<00:00, 38.16it/s]


In [42]:
query

'How do I execute a command in a running docker container?'

In [47]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}
  

In [48]:
query = 'How do I execute a command in a running docker container?'

In [49]:
response = es_client.search(index=index_name, body=search_query)

In [50]:
response['hits']['hits']

[{'_index': 'course-questions',
  '_id': '15xURpABQoM1UiTogUOL',
  '_score': 84.220634,
  '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': 'i5xXRpABQoM1UiTo7Efc',
  '_score': 84.220634,
  '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id

In [60]:
context_template = """
Q: {question}
A: {text}
""".strip()

In [72]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION: {question}
    
    CONTEXT:
    Q: {question}
    A: {text}
    """.strip()
    
    context = ""

    for doc in search_results:
        context = context + f"section:  {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n" 

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [73]:
prompt = build_prompt(query, search_results)

KeyError: 'text'

In [70]:
prompt

"You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. \n    Use only the facts from the CONTEXT when answering the QUESTION.\n    \n    QUESTION: How do I execute a command in a running docker container?\n    \n    CONTEXT:\n    section:  Module 1: Docker and Terraform\nquestion: PGCLI - running in a Docker container\nanswer: In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker container instead.\nBelow the usage with values used in the videos of the course for:\nnetwork name (docker network)\npostgres related variables for pgcli\nHostname\nUsername\nPort\nDatabase name\n$ docker run -it --rm --network pg-network ai2ys/dockerized-pgcli:4.0.1\n175dd47cda07:/# pgcli -h pg-database -U root -p 5432 -d ny_taxi\nPassword for root:\nServer: PostgreSQL 16.1 (Debian 16.1-1.pgdg120+1)\nVersion: 4.0.1\nHome: http://pgcli.com\nroot@pg-database:ny_taxi> \\dt\n+--------+----------------

In [67]:
len(prompt)

3908

In [53]:
search_results = elastic_search(query)

In [71]:
search_results

[{'text': 'In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker container instead.\nBelow the usage with values used in the videos of the course for:\nnetwork name (docker network)\npostgres related variables for pgcli\nHostname\nUsername\nPort\nDatabase name\n$ docker run -it --rm --network pg-network ai2ys/dockerized-pgcli:4.0.1\n175dd47cda07:/# pgcli -h pg-database -U root -p 5432 -d ny_taxi\nPassword for root:\nServer: PostgreSQL 16.1 (Debian 16.1-1.pgdg120+1)\nVersion: 4.0.1\nHome: http://pgcli.com\nroot@pg-database:ny_taxi> \\dt\n+--------+------------------+-------+-------+\n| Schema | Name             | Type  | Owner |\n|--------+------------------+-------+-------|\n| public | yellow_taxi_data | table | root  |\n+--------+------------------+-------+-------+\nSELECT 1\nTime: 0.009s\nroot@pg-database:ny_taxi>',
  'section': 'Module 1: Docker and Terraform',
  'question': 'PGCLI - running in a Docker container',
 